In [1]:
from Pegasus.api import * 
from pathlib import Path

In [2]:
PATHS = {
            #'test1' : Path('.').resolve()/'test1.csv',
            #'test2' : Path('.').resolve()/'test2.csv'
            'MMD_INFO' : Path('.').resolve()/"CrisisMMD/tweets/MMD_informative.csv",
            'MMD_NONINFO' : Path('.').resolve()/"CrisisMMD/tweets/MMD_non_informative.csv",
            'LEX_INFO' : Path('.').resolve()/"CrisisLexT26/tweets/Lex_informative.csv",
            'LEX_NONINFO' : Path('.').resolve()/"CrisisLexT26/tweets/Lex_non_informative.csv"
        }

input_files = {}
output_files = {}
jobs = {}

In [3]:
rc = ReplicaCatalog()
for key, csv in PATHS.items():
    name = str(csv).split("/")[-1]
    input_files[key] = [File(name), name]
    rc.add_replica("local", input_files[key][0], csv)

rc.write()

In [4]:
tc = TransformationCatalog()

preprocess = Transformation(
                "preprocess",
                site="local",
                pfn="/home/scitech/shared-data/crisis-workflow/preprocess.py",
                is_stageable=True
)

tc.add_transformations(preprocess)
tc.write()

In [5]:
wf = Workflow("pipeline")

for k, v in input_files.items():
    output_files[k] = File("processed_"+v[1])
    jobs["Job_"+k] = Job("preprocess")\
                        .add_args('--i', v[0], '--o', output_files[k])\
                        .add_inputs(v[0])\
                        .add_outputs(output_files[k])

for k,v in jobs.items():
    wf.add_jobs(v)
wf.write()

In [6]:
try:
    wf.plan(submit=True)\
        .wait()
except PegasusClientError as e:
    print(e)

2021.02.07 08:33:29.276 UTC:
2021.02.07 08:33:29.281 UTC:   -----------------------------------------------------------------------
2021.02.07 08:33:29.287 UTC:   File for submitting this DAG to HTCondor           : pipeline-0.dag.condor.sub
2021.02.07 08:33:29.292 UTC:   Log of DAGMan debugging messages                 : pipeline-0.dag.dagman.out
2021.02.07 08:33:29.297 UTC:   Log of HTCondor library output                     : pipeline-0.dag.lib.out
2021.02.07 08:33:29.303 UTC:   Log of HTCondor library error messages             : pipeline-0.dag.lib.err
2021.02.07 08:33:29.308 UTC:   Log of the life of condor_dagman itself          : pipeline-0.dag.dagman.log
2021.02.07 08:33:29.313 UTC:
2021.02.07 08:33:29.318 UTC:   -no_submit given, not submitting DAG to HTCondor.  You can do this with:
2021.02.07 08:33:29.329 UTC:   -----------------------------------------------------------------------
2021.02.07 08:33:30.316 UTC:   Your database is compatible with Pegasus version: 5.1.0dev
20

[####################################] 100.0% ..Success (Completed: 12, Queued: 0, Running: 0, Failed: 0)
